# ДЗ №4

Чтобы было больше времени на выполнение курсовой работы, задание выполнить на наборе данных для соревнования:

## Загрузка / подготовка

In [224]:
import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import catboost as cb
import xgboost as xgb
import lightgbm as lb

from scipy.stats import rankdata, gmean
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

pd.set_option("display.max_columns", 30)

In [193]:
def get_input(data_path: str) -> pd.DataFrame:
    base_path = "../kurs/data"
    data = pd.read_csv(f"{base_path}/{data_path}.zip", compression='zip')
    data.columns = [col.lower() for col in data.columns]
    print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")

    return data


def cross_validation(model_class, params, X, y, cv):
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])
    rf_flag = params.get('rf_flag', False)
    if rf_flag:
        del params['rf_flag']

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        pp = {}
        if rf_flag:
            x_train = x_train.fillna(0)
            x_valid = x_valid.fillna(0)
        else:
            pp['eval_set'] = [(x_train, y_train), (x_valid, y_valid)]
            pp['verbose'] = None

        model = model_class(**params)
        model.fit(x_train, y_train, **pp)

        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"   Fold {fold+1}, Valid score = {round(score, 5)}")

        folds_scores.append(round(score, 5))
        estimators.append(model)

    return estimators, oof_preds


def getOOF(models, train, target, test, cv):
    train_pred = pd.DataFrame(index=train.index)
    test_pred = pd.DataFrame(index=test.index)
    
    for name,klass,params in models:
        print(f'============  Train {name} ============')
        m, p = cross_validation(klass, params, train, target, cv)
        tst = test if params.get('rf_flag', False) else test.fillna(0)
        res = [list(md.predict_proba(tst)[:, 1]) for md in m]
        train_pred[name] = p
        test_pred[name] = np.array(res).mean(axis=0)

    return train_pred, test_pred

In [25]:
train = get_input("train.csv")
test = get_input("test.csv")
client_profile = get_input("client_profile.csv")

data = pd.concat([train, test], axis=0)
data = data.reset_index(drop=True)
data = data.merge(
    client_profile, how="left", on="application_number"
)

categorial = data.dtypes[data.dtypes == "object"].index
numerical = list(set(data.columns) - set(categorial))
data[numerical] = data[numerical].astype(float)
data[categorial] = data[categorial].astype('category')

data = pd.get_dummies(data)

mask = data["target"].isnull()
train, test = data.loc[~mask], data.loc[mask]

target = train["target"]
train = train.drop(["application_number", "target"], axis=1)
test = test.drop(["application_number", "target"], axis=1)

data.head(3)

train.csv: shape = 110093 rows, 3 cols
test.csv: shape = 165141 rows, 2 cols
client_profile.csv: shape = 250000 rows, 24 cols


,application_number,target,childrens,total_salary,amount_credit,amount_annuity,region_population,age,days_on_last_job,own_car_age,flag_phone,flag_email,family_size,external_scoring_rating_1,external_scoring_rating_2,...,name_contract_type_Credit Card,gender_F,gender_M,gender_XNA,education_level_Academic degree,education_level_Higher education,education_level_Incomplete higher,education_level_Lower secondary,education_level_Secondary / secondary special,family_status_Civil marriage,family_status_Married,family_status_Separated,family_status_Single / not married,family_status_Unknown,family_status_Widow
0,123687442.0,0.0,1.0,157500.0,855000.0,25128.0,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,...,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0
1,123597908.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,123526683.0,0.0,0.0,135000.0,1006920.0,42660.0,0.026392,21557.0,3618.0,NaN,1.0,0.0,2.0,NaN,0.682149,...,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0


## Обучить алгоритмы LightGBM и XGBoost и CatBoost

* получить OOF прогнозы оценить корреляцию прогнозов на обучающей выборке. 
* Применить модели на тестовую выборку и оценить корреляцию.

In [280]:
n_estimators = 50
early_stop = 50
random_state = 42
eta = 0.3

cv = KFold(n_splits=10, random_state=random_state, shuffle=True)

models = (
    (
        'LightGBM',
        lb.LGBMClassifier,
        {
            "eval_metric": "auc",
            "learning_rate": eta,
            "n_estimators": n_estimators,
            "early_stopping_rounds": early_stop,
            "random_state": random_state,
        }
    ),
    (
        'XGBoost',
        xgb.XGBClassifier,
        {
            "eval_metric": "auc",
            "booster": "gbtree",
            "objective": "binary:logistic",
                
            "learning_rate": eta,
            "n_estimators": n_estimators,
            "early_stopping_rounds": early_stop,
            "random_state": random_state,
        }
    ),
    (
        'CatBoost',
        cb.CatBoostClassifier,
        {
            "eval_metric": "AUC",
            "loss_function": "Logloss",
            'silent': True,
         
            "learning_rate": eta,
            "n_estimators": n_estimators,
            "early_stopping_rounds": early_stop,
            "random_seed": random_state
        }
    ),
    (
        'RForest',
        RandomForestClassifier,
        {
            'rf_flag': True,
            'max_depth': 5,
            "n_estimators": 200,
            "random_state": random_state
        }
    ) 
)

In [281]:
tr, te = getOOF(models, train, target, test, cv)

============  Train LightGBM ============
   Fold 1, Valid score = 0.72227
   Fold 2, Valid score = 0.71586
   Fold 3, Valid score = 0.70108
   Fold 4, Valid score = 0.69707
   Fold 5, Valid score = 0.71467
   Fold 6, Valid score = 0.70817
   Fold 7, Valid score = 0.71053
   Fold 8, Valid score = 0.70389
   Fold 9, Valid score = 0.70054
   Fold 10, Valid score = 0.69857
============  Train XGBoost ============
   Fold 1, Valid score = 0.73598
   Fold 2, Valid score = 0.72794
   Fold 3, Valid score = 0.71766
   Fold 4, Valid score = 0.71736
   Fold 5, Valid score = 0.73073
   Fold 6, Valid score = 0.71778
   Fold 7, Valid score = 0.72313
   Fold 8, Valid score = 0.71511
   Fold 9, Valid score = 0.711
   Fold 10, Valid score = 0.70628
============  Train CatBoost ============
   Fold 1, Valid score = 0.73545
   Fold 2, Valid score = 0.72765
   Fold 3, Valid score = 0.7175
   Fold 4, Valid score = 0.71606
   Fold 5, Valid score = 0.73084
   Fold 6, Valid score = 0.71638
   Fold 7, Valid s

In [282]:
tr.corr()

,LightGBM,XGBoost,CatBoost,RForest
LightGBM,1.000000,0.842765,0.819388,0.718025
XGBoost,0.842765,1.000000,0.937406,0.835303
CatBoost,0.819388,0.937406,1.000000,0.852491
RForest,0.718025,0.835303,0.852491,1.000000


In [283]:
te.corr()

,LightGBM,XGBoost,CatBoost,RForest
LightGBM,1.000000,0.969472,0.767145,0.446601
XGBoost,0.969472,1.000000,0.711958,0.362553
CatBoost,0.767145,0.711958,1.000000,0.814641
RForest,0.446601,0.362553,0.814641,1.000000


**Вывод:**

* при условии похожих параметров обучения результаты по моделям мне кажется можно считать достаточно близкими и скоррелированными
* думаю что XGBoost и LightGBM больше похожи друг на друга а Catboost чуть отличается от них
* RandomForest отличается существенно
* но возможно я не совсем понял что нужно было сделать :) 

## Усреднить прогнозы 
с помощью арифмитического среднего, геометрического среднего и усреднить ранги, сделать выводы о качестве отдельных моделей и о качестве комбинации.

In [284]:
mds = ['LightGBM','XGBoost','CatBoost']
for i in mds:
    score = roc_auc_score(target, tr[i])
    print(f'Model {i:10}: {score:0.5f}')

yhat = tr[mds].mean(axis=1)
print(f'MEAN combination: {roc_auc_score(target, yhat):0.5f}')

yhat = gmean(tr[mds], axis=1)
print(f'G-MEAN combination: {roc_auc_score(target, yhat):0.5f}')

rn = tr[mds].copy()
for i in mds:
    rn[i] = rankdata(rn[i], 'max')

yhat = rn.mean(axis=1)
yhat /= yhat.max()

print(f'RANK combination: {roc_auc_score(target, yhat):0.5f}')

Model LightGBM  : 0.70651
Model XGBoost   : 0.71969
Model CatBoost  : 0.71906
MEAN combination: 0.71972
G-MEAN combination: 0.72066
RANK combination: 0.72068


**Вывод:**

* Думаю в основном данные модели похожи между собой наверное остальное достигается уже тюнингом и конечно фичами данных
* Можно сделать вывод что комбинация посредством усредненения в любом случае лучше одной модели
* Среди комбинаций можно выделить вариант усреднения с помощью геометрического среднего

## (опция) Объединить OOF-прогнозы для трех моделей 

и обучить алгоритм Логистической регрессии (и любой другой, на ваше усмотрение). Сделать выводы о достигаемом качестве, сравнить достигаемое качество с качеством отдельных моделей и моделей, полученных в п.2 и п.4.

**(опция) Обучить алгоритм RandomForest**

(желательно подтюнить параметры) и добавить к построенным ранее моделям. Выполнить задание 5.

In [285]:
def logres(mds, tr, target):
    lr = LogisticRegression(C=0.001, random_state=random_state)

    lr.fit(tr[mds], target)
    yhat = lr.predict_proba(tr[mds])[:, 1]

    return roc_auc_score(target, yhat)

print(f'LogisticRegression: {logres(mds, tr, target):0.5f}')
print(f'LogisticRegression + RandomForest: {logres(mds + ["RForest"], tr, target):0.5f}')

LogisticRegression: 0.71953
LogisticRegression + RandomForest: 0.71957


**Вывод:**

* такой способ позволяет достить качества лучше чем с помощью только одной модели
* но судя по всему за счет получения прогнозов таким способом мы не добьемся лучших результатов чем через усреднение 
* добавление RandomForest не всегда может принести качественные результаты - только если она действительно будет затюненная очень сильно